# TP : Essaims de nano-satellites
Ce TP a pour objectif de vous familiariser davantage avec les topologies d'essaims de nano-satellites en exploitant un jeu de données. Vous verrez notamment qu'on peut extraire énormément d'informations à partir d'un jeu de données simple (ici, positions des satellites par pas de temps). Le TP se déroule en 4 parties :
***
1. Chargement des données : découverte du jeu de données et formattage pour la suite
2. Visualisation de la topologie
3. Caractérisation du réseau : calcul de métriques
4. Partage de charge : division en sous-réseaux
***

Nous allons d'abord importer les librairies nécessaires, dont le module de simulation `swarm_sim` disponible dans le dossier `tp`.

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from swarm_sim import *

## 1. Chargement des données
Les données sont stockées dans le dossier `data` du repository Git, et sont réparties en 100 traces de nano-satellites sur 10000 échantillons.

Ouvrez une trace en exécutant la cellule suivante, et rappelez les champs ainsi que les unités utilisées (n'hésitez pas à revoir le cours, slide 20).

In [ ]:
PATH = '..\data\Traces_Nanosatellites\\track_0.csv' # A adapter en fonction de l'OS

df = pd.read_csv(PATH, sep=',', header=0)
df['coords'] = ['x','y','z']
df = df.set_index('coords', drop=True)

df

Chargez la totalité des données dans ce notebook. Nous allons créer un dictionnaire contenant ces données suivant ce format : 

`satellites[id] = track`

Modifiez la variable `PATH` si nécessaire.

*NB : le chargement peut être assez long (plusieurs minutes en fonction de la machine). Vous pouvez visualiser la progression du chargement avec la fonction `tqdm()` qui fonctionne comme suit :*

In [ ]:
NB_NODES = 100
PATH = '..\data\Traces_Nanosatellites\\track_'

satellites = {}

with tqdm(total=NB_NODES, desc='Extracting data') as pbar:  # On encapsule la boucle en indiquant le nombre max d'itérations
    for i in range(NB_NODES):
        df = pd.read_csv(PATH+str(i)+'.csv', sep=',', header=0)
        df['coords'] = ['x','y','z']
        satellites[i] = df.set_index('coords', drop=True)
        pbar.update(1)

Afin de faciliter la suite du traitement, nous allons créer un objet `Swarm` tel que défini dans le module `swarm_sim` (lire la doc pour le formattage) par pas de temps et les stocker dans un dictionnaire tel que :

`dict_swarm[timestamp] = Swarm`

*NB : le formattage peut également prendre un peu de temps en fonction de la quantité de données à formatter. N'hésitez pas à utiliser la fonction `tqdm()` décrite plus haut si vous le souhaitez.*

In [ ]:
help(Swarm.__init__)

In [ ]:
CONNECTION_RANGE = 30000 # Nous allons travailler avec une portée de communication fixe de 30 km
DURATION = 10000 # Nombre d'échantillons temporels à analyser


dict_swarm = {}

with tqdm(total=DURATION, desc='Converting to Swarm') as pbar:
    for t in range(DURATION):
        dict_swarm[t] = Swarm(
            connection_range=CONNECTION_RANGE, 
            nodes=[Node(id, node[str(t)].x, node[str(t)].y, node[str(t)].z) for id,node in satellites.items()]
        )
        pbar.update(1)

Affichez le contenu du `dict_swarm` à l'instant `0`, ainsi que la description d'un noeud de votre choix. Assurez-vous de bien comprendre tous les champs affichés.

In [ ]:
swarm = dict_swarm[0]

# A faire

## 2. Visualisation de la topologie
Le module `swarm_sim` permet notamment de créer des graphiques 3D représentant les positions des satellites à un instant donné, avec si besoin les ISL (liens inter-satellites) existants. Il s'agit des fonctions `plot_nodes()` et `plot_edges()`.

Affichez la topologie de l'essaim à un pas de temps donné, puis à un autre. 

Qu'observez-vous au niveau de la topologie ?

In [ ]:
dict_swarm[0].plot_nodes()
dict_swarm[0].plot_edges()

Vous avez sûrement remarqué que la fonction `plot_edges()` ne fonctionne pas, ou du moins elle n'affiche aucune connexion dans l'essaim. Pourquoi ?

In [ ]:
""" 
Réponse
"""

Etablissez les connexions entre noeuds voisins grâce à la fonction `neighbor_matrix()` (regardez la doc pour comprendre comment elle fonctionne), puis affichez la topologie de l'essaim grâce à la fonction `plot_edges()`.

In [ ]:
help(Swarm.neighbor_matrix)

In [ ]:
with tqdm(total=DURATION, desc='Neighbor matrix') as pbar:
    for t in range(DURATION):
        neighbor_matrix = dict_swarm[t].neighbor_matrix()
        pbar.update(1)

In [ ]:
dict_swarm[0].plot_edges()

## 3. Caractérisation du réseau
Nous allons maintenant déterminer les caractéristiques du réseau en calculant certaines métriques. Cette caractérisation permet de comprendre plus en détail les différentes dynamiques dans l'essaim, et mettre en avant différents modèles de mobilité. 

La métrique la plus simple est le degré, soit le nombre de voisins directs d'un noeud. Calculez le degré moyen de l'essaim à l'instant `0` (faites bien attention au format des données renvoyées par la fonction `degree()`), puis affichez l'évolution de ce degré moyen sur une révolution complète (c'est-à-dire `1800 échantillons`). Vérifiez que vous avez bien établi la connexion sur une révolution complète avant !

In [ ]:
help(Swarm.degree)

In [ ]:
REVOLUTION = 1800

# 1. Analyse du degré à l'instant 0
swarm = dict_swarm[0]
avg_degree = np.mean(swarm.degree())
print(avg_degree)

In [ ]:
# 2. Analyse du degré moyen sur une révolution
avg_degrees = []

for t in range(REVOLUTION):
    avg_degrees.append(np.mean(dict_swarm[t].degree()))
        
# NB: il est plus propre, en Python, d'écrire cette fonction comme suit :
# avg_degrees = [np.mean(swarm.degree()) for swarm in dict_swarm.values()] 


In [ ]:
# Affichez l'évolution du degré moyen sur une période de révolution
fig = plt.figure(figsize=(8,6))

idx = np.arange(len(avg_degrees))
plt.plot(idx, avg_degrees)

plt.title('Evolution du degré moyen (portée : '+str(CONNECTION_RANGE)+' m)')
plt.xlim(0,REVOLUTION)
plt.ylim(0,15)
plt.xlabel('Temps')
plt.ylabel('Degré moyen')

De la même façon, vous pouvez à présent calculer et regarder l'évolution de la k-vicinity, c'est-à-dire le voisinnage étendu à k sauts, de l'essaim. La fonction correspondante est `k_vicinity()`, attention au format des données renvoyées. Calculez la k-vicinity pour `k = {1,2,3}`.

In [ ]:
help(Swarm.k_vicinity)

In [ ]:
depth_search = [1]
avg_kvicinities = {}

for k in depth_search:
    with tqdm(total=REVOLUTION, desc='Depth '+str(k)) as pbar:  # Visualiser l'avancement des calculs
        temp = []
        for t in range(REVOLUTION):
            temp.append(np.mean(dict_swarm[t].k_vicinity(depth=k)))
            pbar.update(1)
        avg_kvicinities[k] = temp

In [ ]:
fig = plt.figure(figsize=(8,6))

for k, kvicinity in avg_kvicinities.items():
    plt.plot(kvicinity, label='Depth='+str(k))

plt.legend()
plt.title('Evolution de la k-vicinity (portée : '+str(CONNECTION_RANGE)+' m)')
plt.xlabel('Temps')
plt.ylabel('# Voisins')
plt.xlim(0,REVOLUTION)

Que pouvez-vous conclure sur le comportement de la connectivité moyenne (directe et indirecte) du réseau dans le temps ?

In [ ]:
""" 
Réponse
"""

Qu'en est-il de la `distribution` des degrés ?

Tracez, par la méthode de votre choix, la distribution des degrés à trois instants : connectivité minimale (degré moyen minimal), connectivité maximale et connectivité intermédiaire.

In [ ]:
# Trouvez les timestamps correspondant aux niveaux de connectivité recherchés.
topology_min = dict_swarm[0]
topology_max = dict_swarm[0]
topology_avg = dict_swarm[0]

# Tracez les graphiques correspondants (à faire)


Que pouvez-vous en conclure sur la distribution de ces degrés ? Et sur la connectivité globale de l'essaim ?

In [ ]:
""" 
Réponse
"""

## 4. Partage de charge sur le réseau
Nous allons maintenant nous intéresser à la gestion de la redondance et du recouvrement dans le réseau (slides 29+ du cours).

Considérons une mission d'interférométrie pour laquelle nous déployons l'essaim de 100 nanosatellites défini par nos données. Lors de la phase d'observation, i.e. lorsque la Lune bloque la quasi totalité des interférences radio terrestres, chaque nanosatellite va collecter près de 5 Gb de données brutes de l'espace. Afin de créer l'image globale collectée par l'essaim, toutes les données collectées doivent être échangées entre l'intégralité des satellites. Sans politique de limitation de recouvrement, le réseau va saturer très vite.

Afin de limiter ce recouvrement, une solution possible est de diviser le réseau en plusieurs sous-réseaux grâce à des algorithmes de division de graphe **(Graph Division)**. En effet, les sous-réseaux obtenus vont faire office de "noeuds" dans le graphe simplifié, et on va ainsi limiter la quantité de données échangées simultanément. Nous allons analyser les performances de trois algorithmes: **Random Node Division** (`RND`), **Multi-Dimensional Random Walk** (`MDRW`) et **Forest Fire Division** (`FFD`).

Cherchez les fonctions correspondantes à ces algorithmes dans la doc, et assurez-vous de bien comprendre leur principe de fonctionnement. Quelles sont les différences majeures dans leur implémentation ? 

In [ ]:
""" 
Réponse
"""

Prenez un essaim à un instant T et appliquez-lui d'abord l'algorithme `RND`. Paramétrez-le de sorte à obtenir `5` groupes.

In [ ]:
help(Swarm.RND)

In [ ]:
T = 0
NB_GROUPS = 1
swarm = dict_swarm[T]

In [ ]:
# Bonne pratique avant de commencer : on remet tous les noeuds à leur valeur de groupe par défaut (-1)
swarm.reset_groups() 
swarms_rnd = swarm.RND(n=NB_GROUPS)

for i,sw in swarms_rnd.items():
    print(sw)
    print([n.id for n in sw.nodes])

Faites maintenant la même chose mais avec l'algorithme `MDRW` afin d'obtenir une autre répartition.

In [ ]:
help(Swarm.MDRW)

In [ ]:
swarm.reset_groups()
swarms_mdrw = swarm.MDRW(n=NB_GROUPS) 

for i,sw in swarms_mdrw.items():
    print(sw)
    print([n.id for n in sw.nodes])

Enfin, obtenez une troisième répartition grâce à l'algorithme `FFD`.

In [ ]:
help(Swarm.FFD)

In [ ]:
swarm.reset_groups()
swarms_ffd = swarm.FFD(n=NB_GROUPS)

for i,sw in swarms_ffd.items():
    print(sw)
    print([n.id for n in sw.nodes])

A présent, nous allons comparer ces algorithmes en fonction de la répartition de la **taille des groupes** obtenus, l'idéal étant d'obtenir la répartition la plus homogène possible.

Exécutez les cellules suivantes afin de générer les graphiques correspondant aux répartitions des trois algorithmes. Adaptez le nom des variables si besoin.

In [ ]:
distrib_rnd = [len(sw.nodes) for sw in swarms_rnd.values()] # Distribution du nombre de noeuds par groupe
distrib_mdrw = [len(sw.nodes) for sw in swarms_mdrw.values()]
distrib_ffd = [len(sw.nodes) for sw in swarms_ffd.values()]

values = [] # Variable utilisée pour comparer les trois distributions sur la même échelle
values.extend(distrib_rnd)
values.extend(distrib_mdrw)
values.extend(distrib_ffd)

In [ ]:
labels = sorted(set(values))
x_pos = np.arange(min(labels), max(labels)+1)
data_rnd, data_mdrw, data_ffd = [], [], []
for k in x_pos:
       a,b,c = 0,0,0
       if k in distrib_rnd:
            a = len([e for e in distrib_rnd if e==k])
       data_rnd.append(a)
       if k in distrib_mdrw:
            b = len([e for e in distrib_mdrw if e==k])
       data_mdrw.append(b)
       if k in distrib_ffd:
            c = len([e for e in distrib_ffd if e==k])
       data_ffd.append(c)

# Création des histogrammes pour la comparaison 
fig, axes = plt.subplots(nrows=3, figsize=(12,12))
ax = axes[0] # Histogramme RND
ax.bar(x_pos, data_rnd,
       align='center',
       alpha=0.5)
ax.set_ylabel('# Occurrences')
ax.set_xticks(x_pos)
ax.set_xticklabels(x_pos)
ax.set_ylim(0, NB_GROUPS)
ax.set_title('RND distribution')
ax.yaxis.grid(True)

ax = axes[1] # Histogramme MDRW 
ax.bar(x_pos, data_mdrw,
       align='center',
       alpha=0.5)
ax.set_ylabel('# Occurrences')
ax.set_xticks(x_pos)
ax.set_xticklabels(x_pos)
ax.set_ylim(0, NB_GROUPS)
ax.set_title('MDRW distribution')
ax.yaxis.grid(True)

ax = axes[2] # Histogramme FFD 
ax.bar(x_pos, data_ffd,
       align='center',
       alpha=0.5)
ax.set_xlabel('# Nodes in group')
ax.set_ylabel('# Occurrences')
ax.set_xticks(x_pos)
ax.set_xticklabels(x_pos)
ax.set_ylim(0, NB_GROUPS)
ax.set_title('FFD distribution')
ax.yaxis.grid(True)

Ici, nous avons effectué une division à `20%` (5 groupes). Quel algorithme semble le plus adapté dans ce cas ? Le moins adapté ? Pourquoi ?

In [ ]:
""" 
Réponse
"""

Répétez les opérations précédentes, mais cette fois-ci en effectuant une division à `25%` (4 groupes), `10%` (10 groupes), puis `5%` (20 groupes). 

*NB : pour être rigoureux, il faudrait répéter chaque expérience un grand nombre de fois, car les deux algorithmes se basent sur de l'aléatoire (d'où le paramètre "seed" dans les fonctions). Si cela est trop long à réaliser, vous pouvez choisir des seeds différentes de vos voisins afin de confronter vos résultats.*

In [ ]:
# A faire

D'après vos résultats et ceux de vos voisins, comment évolue la performance des algorithmes lorsqu'on augmente le nombre de groupes ? Qu'en concluez-vous sur la performance globale de ces algorithmes sur la division d'un essaim de nanosatellites ?

In [ ]:
""" 
Réponse
"""

### Pour aller plus loin : estimation de la charge réseau

Comme énoncé dans le cours (slides 31-32), la division en groupes a un fort impact sur le nombre de paquets de données à émettre dans l'essaim. La charge du réseau est estimée par la fonction `network_load()` définie ci-dessous, et est égale à la somme des paquets à échanger dans le groupe (`total_intra`) et avec les autres groupes (`total_inter`).

Cette fonction considère 2 cas :
 * situation d'équité (`fairness`) : les noeuds sont répartis équitablement au sein des groupes
 * situation de non-équité : le nombre de noeuds varie en fonction du groupe.

 Compléter cette fonction afin de considérer le 2e cas.

In [ ]:
def network_load(swarm, nb_groups, fairness=True):
    total_nodes = len(swarm.nodes)
    
    if fairness:    
        nodes_per_group = total_nodes/nb_groups
        total_intra = nb_groups * nodes_per_group*(nodes_per_group-1) # Chque membre doit recevoir les paquets de tous les autres membres
    else:
        # A compléter
        pass
    
    total_inter = nb_groups*(nb_groups-1)  # Ce nombre n'est pas affecté par l'équité
    return total_intra + total_inter 

Testez cette fonction sur la topologie (divisée) de votre choix : calculez le nombre de paquets à transmettre relatif à la division de graphe en situation réelle et en situation d'équité.

In [ ]:
# A faire

On peut assez vite comprendre pourquoi on cherche à effectuer une division équitable !

Exécutez la cellule suivante pour visualiser l'évolution de la charge réseau, toujours en situation d'équité, pour différents nombres de groupes.

In [ ]:
nb_groups = [1,2,4,5,10,20,25,50,100]
net_loads = []

for k in nb_groups:
    net_loads.append(network_load(swarm, k, fairness=True))

plt.figure(figsize=(6,6))
plt.plot(nb_groups, net_loads)
plt.xlabel('Number of groups')
plt.ylabel('Number of packets')
plt.xlim(1,100)
plt.ylim(0,10000)

Quelle est la division optimale en termes de charge réseau ? Que se passe-t-il en situation réelle ? Faites le test avec les 3 algorithmes de division et comparez vos résultats.

In [ ]:
# A faire